In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import load_model
import shutil

In [2]:
# ---- CHANGE THIS TO YOUR DATASET PATH ----
BASE_PATH = r"Dataset for Crop Pest and Disease Detection/CCMT Dataset-Augmented"       # folder containing Maize, Tomato, Cashew, Cassava

# folders that contain your inputs
datasets = ["Maize", "Tomato", "Cashew", "Cassava"]

OUTPUT_TRAIN = os.path.join(BASE_PATH, "GlobalDataset", "train")
OUTPUT_TEST = os.path.join(BASE_PATH, "GlobalDataset", "test")

In [3]:
os.makedirs(OUTPUT_TRAIN, exist_ok=True)
os.makedirs(OUTPUT_TEST, exist_ok=True)

In [4]:
for crop in datasets:
    crop_path = os.path.join(BASE_PATH, crop)

    train_path = os.path.join(crop_path, "train_set")
    test_path = os.path.join(crop_path, "test_set")

    print(f"Processing: {crop}")
    
    # ---- TRAIN FOLDER ----
    for disease in os.listdir(train_path):
        old_folder = os.path.join(train_path, disease)

        new_class_name = f"{crop}_{disease}"      # Example: Maize_Blight
        new_folder = os.path.join(OUTPUT_TRAIN, new_class_name)
        os.makedirs(new_folder, exist_ok=True)

        for img in os.listdir(old_folder):
            shutil.copy(os.path.join(old_folder, img), new_folder)
            # ---- TEST FOLDER ----
    for disease in os.listdir(test_path):
        old_folder = os.path.join(test_path, disease)

        new_class_name = f"{crop}_{disease}"
        new_folder = os.path.join(OUTPUT_TEST, new_class_name)
        os.makedirs(new_folder, exist_ok=True)

        for img in os.listdir(old_folder):
            shutil.copy(os.path.join(old_folder, img), new_folder)

print("✅ Global Dataset Ready!")

Processing: Maize
Processing: Tomato
Processing: Cashew
Processing: Cassava
✅ Global Dataset Ready!


In [2]:
# --- 1. Define the Final, Correct Paths ---
# This path goes through all the nested folders to find the data.
train_dir = 'Dataset for Crop Pest and Disease Detection/CCMT Dataset-Augmented/GlobalDataset/train'
test_dir = 'Dataset for Crop Pest and Disease Detection/CCMT Dataset-Augmented/GlobalDataset/test'

IMG_HEIGHT = 224   
IMG_WIDTH = 224
BATCH_SIZE = 32

# --- 2. Create Data Generators ---
print("📦 Setting up Training & Testing Data Generators...")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
print("\n✅ Classes found:", train_generator.class_indices)

num_classes = train_generator.num_classes
print("Total Classes:", num_classes)

📦 Setting up Training & Testing Data Generators...
Found 80271 images belonging to 22 classes.
Found 24981 images belonging to 22 classes.

✅ Classes found: {'Cashew_anthracnose': 0, 'Cashew_gumosis': 1, 'Cashew_healthy': 2, 'Cashew_leaf miner': 3, 'Cashew_red rust': 4, 'Cassava_bacterial blight': 5, 'Cassava_brown spot': 6, 'Cassava_green mite': 7, 'Cassava_healthy': 8, 'Cassava_mosaic': 9, 'Maize_fall armyworm': 10, 'Maize_grasshoper': 11, 'Maize_healthy': 12, 'Maize_leaf beetle': 13, 'Maize_leaf blight': 14, 'Maize_leaf spot': 15, 'Maize_streak virus': 16, 'Tomato_healthy': 17, 'Tomato_leaf blight': 18, 'Tomato_leaf curl': 19, 'Tomato_septoria leaf spot': 20, 'Tomato_verticulium wilt': 21}
Total Classes: 22


In [3]:
input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)

base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False   # Stage 1 – freeze weights


In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 112, 112, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 112, 112, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 112, 112, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 112, 112, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 112, 112, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 112, 112, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 112, 112, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 112, 112, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 113, 113, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 56, 56, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,591,574 (9.89 MB)

 Trainable params: 333,590 (1.27 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
checkpoint = ModelCheckpoint(
    'best_mobilenet_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

earlystop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

In [7]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    callbacks=[checkpoint, earlystop, reduce_lr]
)

C:\Users\HP\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5448 - loss: 1.4061     
Epoch 1: val_accuracy improved from None to 0.67111, saving model to best_mobilenet_model.keras
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 6370s 3s/step - accuracy: 0.6091 - loss: 1.1523 - val_accuracy: 0.6711 - val_loss: 0.9191 - learning_rate: 0.0010
Epoch 2/10
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6633 - loss: 0.9542   
Epoch 2: val_accuracy improved from 0.67111 to 0.70598, saving model to best_mobilenet_model.keras
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 4007s 2s/step - accuracy: 0.6679 - loss: 0.9422 - val_accuracy: 0.7060 - val_loss: 0.8142 - learning_rate: 0.0010
Epoch 3/10
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6777 - loss: 0.9016     
Epoch 3: val_accuracy did not improve from 0.70598
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 4560s 2s/step - accuracy: 0.6820 - loss: 0.8947 - val_accuracy: 0.7005 - val_loss: 0.8305 - learning_rate: 0.0010
Epoch 4/10
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [7]:
model = load_model("best_mobilenet_model.keras")

In [8]:
for layer in model.layers[-50:]:
    layer.trainable = True

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6), 
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [10]:
# Verify how many layers are trainable
trainable_count = sum([layer.trainable for layer in model.layers])
print("Total trainable layers:", trainable_count)

Total trainable layers: 51


In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(
    'best_mobilenet_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

earlystop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

In [12]:
fine_tune_history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,                 
    callbacks=[checkpoint, earlystop, reduce_lr]
)


C:\Users\HP\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7657 - loss: 0.6474     
Epoch 1: val_accuracy improved from None to 0.78644, saving model to best_mobilenet_model.keras
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 8887s 4s/step - accuracy: 0.7659 - loss: 0.6482 - val_accuracy: 0.7864 - val_loss: 0.5903 - learning_rate: 1.0000e-06
Epoch 2/10
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7696 - loss: 0.6337     
Epoch 2: val_accuracy improved from 0.78644 to 0.78740, saving model to best_mobilenet_model.keras
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 5805s 2s/step - accuracy: 0.7671 - loss: 0.6408 - val_accuracy: 0.7874 - val_loss: 0.5839 - learning_rate: 1.0000e-06
Epoch 3/10
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7716 - loss: 0.6304     
Epoch 3: val_accuracy improved from 0.78740 to 0.78912, saving model to best_mobilenet_model.keras
2509/2509 ━━━━━━━━━━━━━━━━━━━━ 5751s 2s/step - accuracy: 0.7684 - loss: 0.6367 - val_accuracy: 0.7891 - val_loss: 0.5800 - learning_

In [13]:
best_model = load_model('best_mobilenet_model.keras')
loss, accuracy = best_model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy:.4f}")

781/781 ━━━━━━━━━━━━━━━━━━━━ 749s 953ms/step - accuracy: 0.7970 - loss: 0.5594 
Test Accuracy: 0.7970
